In [14]:
import csv
import numpy as np
import sklearn
import cv2

image0 = []
image1 = []
image2 = []

key = []

with open('/Users/willbuxton/data/data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    path = '/Users/willbuxton/data/'
    for row in reader:
        source0 = row['path0']
        dir0 = path + source0.split('data/')[-1]
        file0 = cv2.imread(dir0)
        source1 = row['path1'] 
        dir1 = path + source1.split('data/')[-1]
        file1 = cv2.imread(dir1)
        #source2 = row['path2']
        #dir2 = path + source2.split('data/')[-1]
        #file2 = cv2.imread(dir2)
        key.append(row['type'])
        image0.append(file0)
        image1.append(file1)
        #image2.append(file2)
with open('/Users/willbuxton/dataCopy/data.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    path = '/Users/willbuxton/dataCopy/'
    for row in reader:
        source0 = row['path0']
        dir0 = path + source0.split('dataCopy/')[-1]
        file0 = cv2.imread(dir0)
        source1 = row['path1'] 
        dir1 = path + source1.split('dataCopy/')[-1]
        file1 = cv2.imread(dir1)
        #source2 = row['path2']
        #dir2 = path + source2.split('data/')[-1]
        #file2 = cv2.imread(dir2)
        key.append(row['type'])
        image0.append(file0)
        image1.append(file1)
        #image2.append(file2)
print("done")

X0 = np.array(image0)
X1 = np.array(image1)
#X2 = np.array(image2)
y = np.array(key)


done


In [16]:
from numpy import save
save("X0", X0)
save("X1", X1)
#save("X2", X2)
save("y", y)

In [55]:
from numpy import load
X0 = load("X0.npy")
X1 = load("X1.npy")
X2 = load("X2.npy")
y = load("y.npy")
X0 = np.concatenate((X0, load("X0-1.npy")), axis=0)
X1 = np.concatenate((X1, load("X1-1.npy")), axis=0)
X2 = np.concatenate((X2, load("X2-1.npy")), axis=0)
y = np.concatenate((y, load("y-1.npy")), axis=0)




In [56]:
print(X0.shape, X1.shape, X2.shape, y.shape)


(11472, 120, 160, 3) (11472, 120, 160, 3) (11472, 120, 160, 3) (11472,)


In [57]:
y_one_hot = []
for val in y:
    if val == 's':
        y_one_hot.append([1, 0, 0, 0])
    if val == '-' or val == 'z':
        y_one_hot.append([0, 1, 0, 0])
    if val == '.' or val == 'x' or val == 'e':
        y_one_hot.append([0, 0, 1, 0])
    if val == ' ':
        y_one_hot.append([0, 0, 0, 1])
y_one = np.array(y_one_hot)
print(y_one.shape)


(11472, 4)


In [58]:
from sklearn.utils import shuffle
X0, X1, X2, y_one = shuffle(X0, X1, X2, y_one)

In [59]:
import keras
from keras.models import Sequential
from keras.layers import *

left = Sequential()
middle = Sequential()
right = Sequential()
together = Sequential()

left.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(120,160,3)))
left.add(Cropping2D(cropping=((0,0), (0,65))))
left.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
left.add(MaxPooling2D())
left.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
left.add(MaxPooling2D())
left.add(Flatten())

right.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(120,160,3)))
right.add(Cropping2D(cropping=((0,0), (65,0))))
right.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
right.add(MaxPooling2D())
right.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
right.add(MaxPooling2D())
right.add(Flatten())

middle.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(120,160,3)))
middle.add(Cropping2D(cropping=((0,0), (32,33))))
middle.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
middle.add(MaxPooling2D())
middle.add(Convolution2D(6, 5, 5,subsample=(2,2), activation='relu'))
middle.add(MaxPooling2D())
middle.add(Flatten())

merged = Merge([left, middle, right], mode='concat')
together.add(merged)
together.add(Dense(120))
together.add(Dense(84))
together.add(Dense(4))

print(together.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_19 (Lambda)               (None, 120, 160, 3)   0           lambda_input_19[0][0]            
____________________________________________________________________________________________________
cropping2d_19 (Cropping2D)       (None, 120, 95, 3)    0           lambda_19[0][0]                  
____________________________________________________________________________________________________
convolution2d_37 (Convolution2D) (None, 58, 46, 6)     456         cropping2d_19[0][0]              
____________________________________________________________________________________________________
maxpooling2d_37 (MaxPooling2D)   (None, 29, 23, 6)     0           convolution2d_37[0][0]           
___________________________________________________________________________________________

In [60]:
together.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
together.fit([X0, X1, X2], y_one, validation_split=0.2, batch_size=32, nb_epoch=5, verbose=1)


Train on 9177 samples, validate on 2295 samples
Epoch 1/5
9177/9177 [==============================] - 62s - loss: 7.2998 - acc: 0.3551 - val_loss: 7.6482 - val_acc: 0.3364

In [61]:
together.save("model.h5")
